In [1]:
!pip install -q numpy pandas scikit-learn joblib
!pip install "qiskit>=1.0.0" qiskit-aer qiskit-machine-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install qiskit-algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.2 MB/s eta 0:00:00


In [6]:
import os
import numpy as np
import pandas as pd
import joblib
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_algorithms.optimizers import SPSA

import warnings
warnings.filterwarnings('ignore')
os.makedirs("Result", exist_ok=True)
start_time = time.time()

# Load Dataset
df = pd.read_csv('/content/drive/My Drive/merged.csv')

# Use selected 20 features + label
features = ["Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
            "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total", "Flow Duration", "Bwd Packet Length Max",
            "Flow IAT Max", "Flow IAT Mean", "Total Length of Bwd Packets", "Fwd Packet Length Min",
            "Bwd Packet Length Mean", "Flow Packets/s", "Fwd Packet Length Mean", "Total Backward Packets",
            "Fwd Packet Length Max", "Total Fwd Packets", "Bwd Packet Length Min", "Label"]

df = df[features].fillna(0)
df['Label'] = df['Label'].apply(lambda x: 1 if x == "BENIGN" else 0)

x = df.drop("Label", axis=1)
y = df["Label"]

# Scale data
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

# PCA
pca = PCA(n_components=4)
x_pca = pca.fit_transform(x_scaled)

# Train-test split: balanced small sample
x_train, x_test, y_train, y_test = train_test_split(
    x_pca, y, train_size=2000, test_size=400, stratify=y, random_state=42
)

# Quantum model
num_qubits = x_pca.shape[1]
feature_map = ZZFeatureMap(num_qubits, reps=2)
ansatz = TwoLocal(num_qubits, ['ry', 'rz'], 'cz', reps=2)

backend = AerSimulator()
sampler = BackendSampler(backend)
optimizer = SPSA(maxiter=60)

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    sampler=sampler
)

print("Training VQC...")
start_time = time.time()
vqc.fit(np.asarray(x_train), np.asarray(y_train))
print(f"Training time: {time.time() - start_time:.2f} sec")

# Predict
y_pred = vqc.predict(np.asarray(x_test))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save
joblib.dump(scaler, "Result/vqc_scaler.pkl")
joblib.dump(pca, "Result/vqc_pca_transform.pkl")
joblib.dump(x.columns.tolist(), "Result/vqc_features_used.pkl")

Training VQC...
Training time: 2343.71 sec

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.30      0.38        67
           1       0.87      0.94      0.90       333

    accuracy                           0.83       400
   macro avg       0.69      0.62      0.64       400
weighted avg       0.81      0.83      0.82       400



['Result/vqc_features_used.pkl']